In [1]:
# we have the assest allocation generated, but how well does it do?

In [2]:
from generalImport import *
from util import *

In [59]:
PriceDict = load_dict("price_of_stocks")
priceMovement = extract_stocks_historical_price_from_dict(PriceDict)
scaledPriceMovement = price_scaling(priceMovement)
weights = generate_portfolio_weights(5)
initialInvestment = 1000000
allocatedDf = asset_allocation(scaledPriceMovement, weights, initialInvestment)
allocatedDf

Loaded 5 DataFrames from 'price_of_stocks'.


,Date,AAPL,JPM,KO,META,S&P500,Portfolio Value [$],Portfolio Daily Return [%]
0,2020-11-30 00:00:00-05:00,206410.811225,135714.293590,296255.384928,71945.443764,289674.066493,1.000000e+06,0.000000
1,2020-12-01 00:00:00-05:00,211358.859176,137262.109150,298693.850753,74508.778699,293511.113552,1.015335e+06,1.533471
2,2020-12-02 00:00:00-05:00,212035.925070,137472.680010,297689.700666,75079.146249,293169.686581,1.015447e+06,0.011073
3,2020-12-03 00:00:00-05:00,213537.702131,137853.941766,300759.271959,74437.989358,294135.451973,1.020724e+06,0.519694
4,2020-12-04 00:00:00-05:00,212140.074464,139407.451702,305894.282153,73787.646872,295347.274111,1.026577e+06,0.573355
...,...,...,...,...,...,...,...,...
1251,2025-11-21 00:00:00-05:00,480652.487345,385185.564669,479056.416155,155669.719599,528247.615771,2.028812e+06,-0.420268
1252,2025-11-24 00:00:00-05:00,488589.044309,385081.913898,479221.983494,160201.373235,534838.923236,2.047933e+06,0.942494
1253,2025-11-25 00:00:00-05:00,495482.270669,388794.517640,480347.760566,165612.961180,538442.720240,2.068680e+06,1.013070
1254,2025-11-26 00:00:00-05:00,495954.896330,396355.823609,481705.316757,167544.355791,545613.077987,2.087173e+06,0.893963


In [ ]:
def measure(assetAllocation, rf = 0.03):
    # in the course they need to input weight, and initial investment, but 
    #   initial investment and weight can be derived from assetAllocation table
    # 1 calculate weight and initial investment.
    # weight and initial investment
    initialInvestment = float(assetAllocation.iloc[0, -2])
    initialWeight = assetAllocation.iloc[0, 1:(-1-1)] / initialInvestment

    # 1: ROI
    # what we get at the end compare with intial
    ROI = (float(assetAllocation.iloc[-1, -2]) / initialInvestment  - 1 ) * 100

    # 2: SHARP
    modiAssetAllocation = assetAllocation.drop(columns=assetAllocation.columns[[0, -1, -2]]).pct_change(1)
        # using the formular provided from the course
    expected_port_return = np.sum(initialWeight * modiAssetAllocation.mean()) * 252
    covariance = modiAssetAllocation.cov() * 252
    expected_volatility = np.sqrt(np.dot(initialWeight.T, np.dot(covariance, initialWeight)))
    shareRatio = (expected_port_return - rf) / expected_volatility
    return {"expectedReturn": float(expected_port_return), "expectedVolatility": float(expected_volatility), "shareRatio": float(shareRatio), "endReturn": float(assetAllocation.iloc[-1, -2]), "returnOnInvestment": ROI} 

In [61]:
# using the instructioned one:

def simulation_engine(weights, initial_investment):
    # Perform asset allocation using the random weights (sent as arguments to the function)
    portfolio_df = asset_allocation(scaledPriceMovement, weights, initial_investment)
  
    # Calculate the return on the investment 
    # Return on investment is calculated using the last final value of the portfolio compared to its initial value
    return_on_investment = ((portfolio_df['Portfolio Value [$]'][-1:] - 
                             portfolio_df['Portfolio Value [$]'][0])/ 
                             portfolio_df['Portfolio Value [$]'][0]) * 100
  
    # Daily change of every stock in the portfolio (Note that we dropped the date, portfolio daily worth and daily % returns) 
    portfolio_daily_return_df = portfolio_df.drop(columns = ['Date', 'Portfolio Value [$]', 'Portfolio Daily Return [%]'])
    portfolio_daily_return_df = portfolio_daily_return_df.pct_change(1) 
  
    # Portfolio Expected Return formula
    expected_portfolio_return = np.sum(weights * portfolio_daily_return_df.mean() ) * 252
  
    # Portfolio volatility (risk) formula
    # The risk of an asset is measured using the standard deviation which indicates the dispertion away from the mean
    # The risk of a portfolio is not a simple sum of the risks of the individual assets within the portfolio
    # Portfolio risk must consider correlations between assets within the portfolio which is indicated by the covariance 
    # The covariance determines the relationship between the movements of two random variables
    # When two stocks move together, they have a positive covariance when they move inversely, the have a negative covariance 

    covariance = portfolio_daily_return_df.cov() * 252 
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

    # Check out the chart for the 10-years U.S. treasury at https://ycharts.com/indicators/10_year_treasury_rate
    rf = 0.03 # Try to set the risk free rate of return to 1% (assumption)

    # Calculate Sharpe ratio
    sharpe_ratio = (expected_portfolio_return - rf)/expected_volatility 
    return expected_portfolio_return, expected_volatility, sharpe_ratio, portfolio_df['Portfolio Value [$]'][-1:].values[0], return_on_investment.values[0]



In [62]:
measure(allocatedDf)

{'expectedReturn': 0.1640051351746843,
 'expectedVolatility': 0.12953680078949442,
 'shareRatio': 1.0344947100588906,
 'endReturn': 2098452.0689471317,
 'returnOnInvestment': 109.8452068947132}

In [53]:
allocatedDf["Portfolio Value [$]"][-1:]

1255    2.057754e+06
Name: Portfolio Value [$], dtype: float64

In [54]:
(allocatedDf["Portfolio Value [$]"][-1:] - allocatedDf["Portfolio Value [$]"][0]) / (allocatedDf['Portfolio Value [$]'][0]) * 100

1255    105.775432
Name: Portfolio Value [$], dtype: float64

In [16]:
ROI

0.010577543150272364

In [17]:
float(assetAllocation.iloc[-1, -2]) 

2057754.3150272365

In [22]:
(float(assetAllocation.iloc[-1, -2]) - initialInvestment) / initialInvestment / 100

0.010577543150272364

In [35]:
expected_port_return

np.float64(0.16033305157788147)

In [36]:
modiAssetAllocation

,AAPL,JPM,KO,META,S&P500
0,NaN,NaN,NaN,NaN,NaN
1,0.023972,0.011405,0.008231,0.035629,0.013246
2,0.003203,0.001534,-0.003362,0.007655,-0.001163
3,0.007083,0.002773,0.010311,-0.008540,0.003294
4,-0.006545,0.011269,0.017073,-0.008737,0.004120
...,...,...,...,...,...
1251,-0.004341,-0.022349,0.018228,-0.008478,-0.009202
1252,0.016512,-0.000269,0.000346,0.029111,0.012478
1253,0.014108,0.009641,0.002349,0.033780,0.006738
1254,0.000954,0.019448,0.002826,0.011662,0.013317


In [38]:
covariance

,AAPL,JPM,KO,META,S&P500
AAPL,0.054988,0.015761,0.006698,0.035027,0.021743
JPM,0.015761,0.046393,0.005665,0.019344,0.018171
KO,0.006698,0.005665,0.017857,0.000662,0.005182
META,0.035027,0.019344,0.000662,0.147979,0.028904
S&P500,0.021743,0.018171,0.005182,0.028904,0.018141


In [ ]:
#